In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import yfinance as yf

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

ModuleNotFoundError: No module named 'yfinance'

In [15]:
ticker = str(input('Please enter the ticker for the equity you wish to find a pair for: ')).upper()

print('\nYou entered {}.'.format(ticker))

Please enter the ticker for the equity you wish to find a pair for:  acls



You entered ACLS.


# Check to see if ticker is part of SP500, if not, add to the list of data to pull

In [5]:
# Read in all SP500 entities
sp500=pd.read_csv('spy_holdings.csv').dropna()

# Build list of available sectors
sector_list = pd.DataFrame(sp500['sector'].unique())
sector_list = sector_list.rename(columns={0:'Sector'})

In [16]:
# Check if stock is in S&P 500 and retrieve sector information
appropos = 0
tick_match = sp500.identifier.eq(ticker).map(int)
if tick_match.sum() > 0:
    print('Your equity is in the S&P500')
    appropos = 1
    tick_cont = 1
    # Isolate security of interest
    tick_data = sp500[sp500['identifier'].str.contains(ticker)]

    # Identify sector in S&P 500 listing
    in_sector = str(tick_data['sector'].iloc[0])
       
# If stock is not in S&P500, Retrieve User Sector Selection
while appropos == 0:
    print("""
    Your stock is not listed in the S&P 500.
    Please select a sector by number from the list below:
           """)
    print(sector_list,'\n\nUser Input: ')
    assign_sector = int(input(""))
        
    # Check that the input was appropriate
    if assign_sector >= 0 and assign_sector < 12:
        appropos = 1
        in_sector = sector_list['Sector'].iloc[assign_sector]
    else:
        print("""That was not an appropriate selection. Plese try again: 
                          """
                         )
# Output    
    print('You have selected {} sector.'.format(in_sector))

print('Finding best pair for {} in {} sector.'.format(ticker,in_sector))               


    Your stock is not listed in the S&P 500.
    Please select a sector by number from the list below:
           
                    Sector
0   Information Technology
1   Consumer Discretionary
2   Communication Services
3               Financials
4              Health Care
5                   Energy
6         Consumer Staples
7              Industrials
8                Utilities
9                Materials
10             Real Estate
11              Unassigned 

User Input: 


 7


You have selected Industrials sector.
Finding best pair for ACLS in Industrials sector.


In [19]:
# Select all securities in the S&P500 from the appropriate sector
sectorized = sp500[sp500['sector'].str.contains(in_sector)]

# Create list of sector tickers
sector_tickers = list(sectorized['identifier'])
if tick_match.sum() > 0:
    pass
else:
    sector_tickers.append(ticker)

In [ ]:
# Download data from yahoo
tick_data = yf.download(sector_tickers,
                        period = "1y",
                        interval = '1m',
                        parse_dates=['Date']
                       )['Adj Close']

In [ ]:
# Compute rolling 30 day returns, return correlation, return covariance
daily_returns = np.log(tick_data.rolling(22).mean() / tick_data.shift(1))*100
corr_daily = daily_returns.corr()
cov_daily = tick_data.cov()

# Select highest covariance and correlation
max_corr = corr_daily[ticker].nlargest(15)
cov = cov_daily.unstack().sort_values(ascending=False)
max_cov = cov[ticker].nlargest(15)

# Merge into a new DF
rel_matrix = pd.DataFrame(max_cov)
rel_matrix = rel_matrix.rename(columns={0:'cov'})
rel_matrix['corr'] = max_corr

# Plot correlation matrix for returns post FB IPO
top_corr_features = corr_daily.index
plt.figure(figsize = (30,30))

sns.heatmap(corr_daily[top_corr_features].corr(), 
            annot = True, 
            cmap = 'RdYlGn', 
            vmin = -1, 
            vmax = 1
           )

plt.figure(figsize = (30,30))
sns.heatmap(tick_data.cov(),
           annot = True,
           cmap = 'RdYlGn'
           )

# Compute Equal Weighted Portfolio

# calculate mu and S
mu = expected_returns.mean_historical_return(tick_data)
s = risk_models.sample_cov(tick_data)

# calculate efficient frontier
ef = EfficientFrontier(mu, s)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
latest_prices = get_latest_prices(eqw['Adj Close'])

da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.lp_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))